<a href="https://colab.research.google.com/github/ysikalie/colorization/blob/master/colorization_densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer,ZeroPadding2D, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as  tf
from keras.applications.densenet import DenseNet201
X = []
for filename in os.listdir('Train/train'):
    X.append(img_to_array(load_img('Train/train/'+filename, target_size=(224, 224))))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X
X_Test = []
for filename in os.listdir('Test/test'):
    X_Test.append(img_to_array(load_img('Test/test/'+filename,target_size=(224, 224))))
X_Test = np.array(X_Test, dtype=float)

Xtest = 1.0/255*X_Test

#Load weights
densenet = DenseNet201(weights=None ,include_top=False, input_shape=(224, 224,1))


[ 0.92787748 -0.04521731]


In [0]:
densenet = Model(inputs=densenet.input, outputs=densenet.get_layer('pool2_conv').output)
densenet.summary()
print('  ' ,densenet.output)

In [0]:
from keras.layers import ZeroPadding2D
from keras import layers,backend
def dense_block(x, blocks, name):
    """A dense block.

    # Arguments
        x: input tensor.
        blocks: integer, the number of building blocks.
        name: string, block label.

    # Returns
        output tensor for the block.
    """
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def transition_block(x, reduction, name):
    """A transition block.

    # Arguments
        x: input tensor.
        reduction: float, compression rate at transition layers.
        name: string, block label.

    # Returns
        output tensor for the block.
    """
    bn_axis =  3
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    x = layers.Conv2D(int(backend.int_shape(x)[bn_axis] * reduction), 1,
                      use_bias=False,
                      name=name + '_conv')(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    """A building block for a dense block.

    # Arguments
        x: input tensor.
        growth_rate: float, growth rate at dense layers.
        name: string, block label.

    # Returns
        Output tensor for the block.
    """
    bn_axis =  3
    x1 = layers.BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + '_0_bn')(x)
    x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = layers.Conv2D(4 * growth_rate, 1,
                       use_bias=False,
                       name=name + '_1_conv')(x1)
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                   name=name + '_1_bn')(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv2D(growth_rate, 3,
                       padding='same',
                       use_bias=False,
                       name=name + '_2_conv')(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x


In [0]:
encode_input =  Input(shape=(224, 224, 1))
encode_output = ZeroPadding2D(padding=((3, 3), (3, 3)))(encode_input)
encode_output = Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(encode_output)
encode_output = BatchNormalization(
    axis=3, epsilon=1.001e-5, name='conv1/bn')(encode_output)
encode_output = Activation('relu', name='conv1/relu')(encode_output)
encode_output = ZeroPadding2D(padding=((1, 1), (1, 1)))(encode_output)
encode_output = MaxPooling2D(3, strides=2, name='pool1')(encode_output)

encode_output = dense_block(encode_output, blocks=6, name='conv2')
encode_output = transition_block(encode_output, 0.5, name='pool2')
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encode_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(8, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encode_input, outputs=decoder_output)

model.summary()

In [0]:
def create_densenet_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (224, 224, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with densenet.graph.as_default():
        embed = densenet.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 2

def train_image_gb_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=2):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128   
        yield (X_batch, Y_batch)

        
def test_image_gb_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128     
        yield (X_batch, Y_batch)
    

model.compile(optimizer='rmsprop', loss='mse')
# model.summary()
model.fit_generator(train_image_gb_gen(batch_size), epochs=2,
                    steps_per_epoch=5,
                    validation_data=train_image_gb_gen(batch_size),
                    validation_steps=4)

In [0]:
color_me = []
for filename in os.listdir('result'):
    if filename.endswith('checkpoints'):
        continue    
    img = load_img('result/'+filename, target_size=(224, 224))             
    color_me.append(img_to_array(img))
color_me = np.array(color_me, dtype=float)
if color_me.shape[-1]==3:
    color_me = rgb2gray(color_me)
color_me_L = np.array(color_me, dtype=float)/ 255.0 * 100 
color_me_L = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me_L)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = color_me_L[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))